In [1]:
# dbutils.library.installPyPI("torch")
# dbutils.library.installPyPI("scikit-learn", version="1.19.1")
# dbutils.library.installPyPI("azureml-sdk", extras="databricks")
# dbutils.library.restartPython()

In [2]:
dbutils.widgets.text("hostName", "defaul value here", "SQL Server Host Name")
dbutils.widgets.text("database", "defaul value here", "Database Name")
dbutils.widgets.text("table", "defaul value here", "Table Name")

In [3]:
storageAccountName = "data lake storage account name"
fileSystemName = "container name here"
appID = "service principal app ID"
secret = "service principal app secret"
tenantID = "AD tenant ID"
mountPoint = "/mnt/covid19"

try:
  dbutils.fs.unmount(mountPoint)
except:

configs = {
  "fs.azure.account.auth.type": "OAuth",
  "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
  "fs.azure.account.oauth2.client.id": appID,
  "fs.azure.account.oauth2.client.secret": secret,
  "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/" + tenantID + "/oauth2/token",
  "fs.azure.createRemoteFileSystemDuringInitialization": "false"
}

dbutils.fs.mount(
  source = "abfss://" + fileSystemName  + "@" + storageAccountName + ".dfs.core.windows.net/covid19",
  mount_point = mountPoint,
  extra_configs = configs
)

/mnt/covid19 has been unmounted.
Out[15]: True

In [4]:
%scala

Class.forName("com.microsoft.sqlserver.jdbc.SQLServerDriver")

res2: Class[_] = class com.microsoft.sqlserver.jdbc.SQLServerDriver

In [5]:
%python

jdbcHostname = dbutils.widgets.get("hostName")
jdbcDatabase = dbutils.widgets.get("database")
jdbcTable = dbutils.widgets.get("table")
jdbcPort = 1433
jdbcUsername = "db user name"
jdbcPassword = "db user password"

jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2}".format(jdbcHostname, jdbcPort, jdbcDatabase)
connectionProperties = {
  "user" : jdbcUsername,
  "password" : jdbcPassword,
  "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

# covid19_df = spark.table("covid19_table")
# covid19_df.write.jdbc(jdbcUrl, table="covid19", mode="overwrite", properties=connectionProperties)

query = "(select * from {0}) mytable".format(jdbcTable)
df = spark.read.jdbc(url=jdbcUrl, table=query, properties=connectionProperties)
# display(df)  

filePath = "{0}/{1}/{2}.csv".format(mountPoint, "/path/", jdbcTable)
df.write.csv(filePath, mode="overwrite", header=True)

filePath = "{0}/{1}/{2}.parquet".format(mountPoint, "/path/", jdbcTable)
df.write.parquet(filePath, mode="overwrite")